In [ ]:
import healpy
import numpy as np
from matplotlib import pyplot as plt
from ch_util import ephemeris as ephem
from caput.time import unix_to_skyfield_time
import h5py
import time

%matplotlib inline
plt.rcParams.update({'figure.figsize': (16, 12), 'font.size': 16})

In [ ]:
haslam_dsds = healpy.fitsfunc.read_map("./lambda_haslam408_dsds.fits")
haslam_nofilt = healpy.fitsfunc.read_map("./lambda_haslam408_nofilt.fits")

In [ ]:
healpy.visufunc.mollview(haslam_dsds)
plt.savefig("/home/tristpinsm/plots/haslam_dsds.pdf", dpi=300)

In [ ]:
healpy.visufunc.mollview(healpy.sphtfunc.smoothing(haslam_dsds, sigma=3e2/408./20./1.95))
plt.savefig("/home/tristpinsm/plots/haslam_dsds_smooth.pdf", dpi=300)

In [ ]:
healpy.visufunc.mollview(haslam_nofilt)

In [ ]:
np.sqrt(haslam_dsds.shape[0]/12)

In [ ]:
class ModelVis(object):
    c = 2.99792458e2
    
    def __init__(self, fname="./lambda_haslam408_dsds.fits", freq=408.):
        # observed frequency
        self.freq = freq
        self.wl = self.c / freq

        # smooth map using CHIME EW primary beam
        self.basemap = healpy.fitsfunc.read_map(fname)
        self.nside = int((self.basemap.shape[0]/12)**0.5)
        self.smoothmap = healpy.sphtfunc.smoothing(self.basemap, sigma=self._res())
        
        # get an observer at CHIME
        self.obs = ephem.chime_observer().skyfield_obs()

    def set_baselines(self, baselines=None):
        if type(baselines) is np.ndarray:
            self.ns_baselines = baselines
        elif type(baselines) is int or type(baselines) is float:
            # generate them
            pass

    def get_vis(self, times, vis, n, max_za=90.):
        # use this to check/visualize model
        self._gen_basis(times, vis, n, max_za)
        return np.sum(self._basis, axis=2)
    
        
    def fit_beam(self, times, vis, weight, n, max_za=90., return_basis=False):
        # generate model basis
        self._gen_basis(times, n, max_za)
        # construct least squares equation
        # take the real part since we omit the lower half of the vis matrix
        M = np.sum(np.matmul(self._basis[:,:,:,np.newaxis], self._basis[:,:,np.newaxis,:]).real
                   * weight[:,:,np.newaxis,np.newaxis], axis=(0,1))
        v = np.sum(((vis * weight)[:,:,np.newaxis] * self._basis).real, axis=(0,1))
        # return solution
        return np.dot(np.linalg.inv(M), v)
        
    def _gen_basis(self, times, vis, n, max_za=90.):
        # evaluate Haslam map at n declinations and all times
        za = np.linspace(-max_za, max_za, n)
        az = np.zeros_like(alt)
        az[:n/2] = 180.
        alt = 90. - np.cos(np.radians(az)) * za
        self.za = za
        self._basis = np.zeros((vis.shape[0], vis.shape[1], za.shape[0]), dtype=np.complex64)
        phases = self._fringe_phase(za)
        for i, t in enumerate(times):
            sf_t = unix_to_skyfield_time(t)
            pos = self.obs.at(sf_t).from_altaz(az_degrees=az, alt_degrees=alt)
            gallat, gallon = pos.galactic_latlon()[:2]
            pix = healpy.ang2pix(self.nside, gallat.radians, gallon.radians, lonlat=True)
            self._basis[:,i,:] = self.smoothmap[pix] * np.exp(1j * phases)
    
    def _res(self):
        # match FWHM of sinc for 20m aperture
        return self.wl / 20. / 1.95
    
    def _fringe_phase(self, za):
        return 2 * np.pi * self.ns_baselines[:,np.newaxis] / self.wl * np.sin(np.radians(za))[np.newaxis,:]

In [ ]:
with h5py.File("/mnt/gong/archive/20180911T015826Z_chimeN2_corr/00004896_0000.h5") as fh:
    f_ind = np.argmin(np.abs(408. - fh['index_map/freq']['centre']))
    vis = fh['vis'][f_ind,:,0:1]
    weight = fh['flags/vis_weight'][f_ind,:,0:1]
    times = fh['index_map/time']['ctime'][0:1]
    gains = fh['gain'][f_ind,:,0:1]
    #stack_prods = fh['index_map/stack']['prod']
    #b = ((fh['index_map/prod']['input_b'][stack_prods] / 512).astype(float)
    #      - fh['index_map/prod']['input_a'][stack_prods] / 512) * 0.3
    b = ((fh['index_map/prod']['input_b'] % 256).astype(float)
         - fh['index_map/prod']['input_a'] % 256) * 0.3
    autos_ind = np.where(fh['index_map/prod']['input_b'][stack_prods]
                         == fh['index_map/prod']['input_a'][stack_prods])
# TODO: apply gains

In [ ]:
# apply gains
vis[:,0] *= np.outer(gains[:,0], gains[:,0].conj())[np.triu_indices(2048)]

In [ ]:
vis[autos_ind,:] = 0.

In [ ]:
test_model = ModelVis()

In [ ]:
test_model.set_baselines(b)

In [ ]:
test_vis = test_model.get_vis(times, vis, 40)

In [ ]:
test_vis

In [ ]:
vis_mat = np.zeros((2048, 2048), dtype=np.complex64)
vis_mat[np.triu_indices(2048)] = test_vis[:,0]
plt.imshow(np.abs(vis_mat), aspect='equal', cmap='magma')
plt.colorbar()
plt.savefig("../plots/haslam_mock_vis.pdf", dpi=300)

In [ ]:
test_soln = test_model.fit_beam(times, vis, weight, 40)

In [ ]:
plt.plot(test_soln)
plt.ylim((-1e9,1e9))

### Sanity check haslam visibilities

In [ ]:
plt.plot(ns_baselines, '.')

In [ ]:
num_t = 4096
num_pix = 512
times = np.linspace(0, 3600. * 24, 4096) + time.time()
test_vis = np.zeros((256*257/2, num_t), dtype=np.complex64)
ns_baselines = (np.arange(256)[np.newaxis,:] - np.arange(256)[:,np.newaxis]).astype(float)[np.triu_indices(256)]
test_model = ModelVis()
test_model.set_baselines(ns_baselines)
for t in range(num_t/64):
    t_slice = slice(t*64, (t+1)*64)
    test_vis[:,t_slice] = test_model.get_vis(times[t_slice], test_vis[:,t_slice], num_pix)

In [ ]:
# back into a map
test_map = np.zeros((num_t, num_pix), dtype=np.complex64)
test_za = np.radians(np.linspace(-90, 90, num_pix))
test_map[:] = np.dot(test_vis.T, np.exp(-2j * np.pi * ns_baselines[:,np.newaxis]
                                        / test_model.wl * np.sin(test_za)[np.newaxis,:]))

In [ ]:
plt.imshow(test_map.T.real, aspect='auto')
plt.colorbar()

In [ ]:
x = np.linspace(-1.5, 1.5, 200)
plt.plot(x, np.sin(np.pi * x)/(np.pi*x))
plt.plot(x, np.exp(-0.5*x**2 / (0.5137)**2))

In [ ]:
# figure out how to get accurate RA/dec, galactic coordinates
ch_obs = ephem.chime_observer()
sf_obs = ch_obs.skyfield_obs()

In [ ]:
test_t = unix_to_skyfield_time(1538961688.)
alt = np.concatenate((np.linspace(0.,90.,100), np.linspace(0.,90.,100)[::-1]))
az = np.zeros_like(alt)
az[:100] = 180.
pos = sf_obs.at(test_t).from_altaz(az_degrees=az, alt_degrees=alt)

In [ ]:
gallat, gallon = pos.galactic_latlon()[:2]

In [ ]:
gallat.degrees